# Muhammad Tahir
## K21-4503
## IR Assignment 01

In [1]:
import os
import json
import re
import tkinter as tk
from nltk import word_tokenize, download
from nltk.corpus import stopwords
from collections import defaultdict

class DocumentIndexer:
    """
    This class is responsible for indexing a set of documents.
    """

    def __init__(self):
        """
        Initializes the DocumentIndexer class.

        - Reads documents from text files.
        - Creates inverted and positional indexes.
        - Saves indexes to files.
        """
        self.num_documents = 22
        self.stopwords_set = set(stopwords.words('english'))

        # Read documents and create document identifiers
        self.document_texts, self.document_identifiers = self.read_documents()

        # Tokenize documents (remove stopwords and punctuation)
        self.document_tokens = self.tokenize_documents(self.document_texts)

        # Create inverted and positional indexes
        self.inverted_index = self.create_inverted_index(self.document_tokens)
        self.positional_index = self.create_positional_index(self.document_tokens)

        # Save indexes to files for persistence
        self.save_indexes_to_files()

    def read_documents(self):
        """
        Reads documents from text files in the "ResearchPapers" folder.

        Returns:
            - document_texts: List of document texts.
            - document_identifiers: List of document identifiers.
        """
        document_texts = []
        document_identifiers = []
        for i in range(1, self.num_documents + 5):
            file_path = os.path.join("ResearchPapers", f"{i}.txt")
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='latin-1') as f:
                    text = f.read()
                    document_texts.append(text)
                    document_identifiers.append(f"Document -> {i}")
        return document_texts, document_identifiers

    def tokenize_documents(self, document_texts):
        """
        Tokenizes documents (breaks text into lowercase words).

        - Removes stop words and punctuation.

        Args:
            document_texts: List of document texts.

        Returns:
            List of tokenized documents (lists of words).
        """
        tokens = []
        for text in document_texts:
            tokenized_text = word_tokenize(text)
            tokenized_text = [word.lower() for word in tokenized_text if (word.isalpha() or word.isdigit()) and word.lower() not in self.stopwords_set]
            tokens.append(tokenized_text)
        return tokens

    def create_inverted_index(self, terms):
        """
        Creates an inverted index for the given terms.

        Args:
            terms: List of tokenized documents.

        Returns:
            Inverted index (defaultdict(list)).
        """
        index = defaultdict(list)
        for i, tokens in enumerate(terms):
            for token in set(tokens):
                index[token].append(i)
        return index

    def create_positional_index(self, tokens):
        """
        Creates a positional index for the given tokens.

        Args:
            tokens: List of tokenized documents.

        Returns:
            Positional index (defaultdict).
        """
        pos_index = defaultdict(lambda: [])
        for doc_id, sub_tokens in enumerate(tokens):
            for term in set(sub_tokens):
                positions = [index for index, element in enumerate(sub_tokens) if element == term]
                pos_index[term].append([doc_id, positions])
        return pos_index

    def save_indexes_to_files(self):
        """
        Saves the inverted and positional indexes to files.

        - Inverted_Index.txt
        - Positional_Index.txt
        """
        with open('Inverted_Index.txt', 'w') as inverted_index_file:
            json.dump(self.inverted_index, inverted_index_file)
        with open("Positional_Index.txt", 'w') as positional_index_file:
            json.dump(self.positional_index, positional_index_file)

class QueryProcessor:
    def __init__(self, indexer):
        """
        Initializes the QueryProcessor class.
        Arguments:
        - indexer: An instance of the DocumentIndexer class.
        Returns: None
        """
        self.indexer = indexer

    def intersection(self, posting1, posting2):
        """
        Performs the intersection of two posting lists.
        Arguments:
        - posting1: First posting list.
        - posting2: Second posting list.
        Returns: List of common elements.
        """
        return list(set(posting1) & set(posting2))

    def union(self, posting1, posting2):
        """
        Performs the union of two posting lists.
        Arguments:
        - posting1: First posting list.
        - posting2: Second posting list.
        Returns: List of unique elements from both lists.
        """
        return list(set().union(posting1, posting2))

    def negate(self, posting1, posting2):
        """
        Performs the negation of posting2 from posting1.
        Arguments:
        - posting1: List to be negated.
        - posting2: List to be subtracted.
        Returns: List of elements in posting1 but not in posting2.
        """
        return list(set(posting1) - set(posting2))

    def process_boolean_query(self, query):
        """
        Processes a boolean query.
        Arguments:
        - query: String representing the boolean query.
        Returns: List of document indices satisfying the query.
        """
        query = query.split()
        term = query[0]
        posting = self.indexer.inverted_index.get(term, [])
        documents = posting
        operation = '&'

        for index in range(1, len(query)):
            if query[index] == "AND":
                operation = '&'
            elif query[index] == "OR":
                operation = '||'
            elif query[index] == "NOT":
                operation = '!'
            else:
                if operation == '&':
                    term = query[index]
                    term = self.indexer.inverted_index.get(term, [])
                    documents = self.intersection(documents, term)
                elif operation == '||':
                    term = query[index]
                    term = self.indexer.inverted_index.get(term, [])
                    documents = self.union(documents, term)
                elif operation == '!':
                    term = query[index]
                    term = self.indexer.inverted_index.get(term, [])
                    documents = self.negate(documents, term)

        return documents

    def positional_intersect(self, posting1, posting2, k):
        """
        Performs positional intersection of two posting lists.
        Arguments:
        - posting1: First positional posting list.
        - posting2: Second positional posting list.
        - k: Proximity threshold.
        Returns: List of document indices satisfying the positional intersection.
        """
        result = []
        if posting1 and posting2:
            len1, len2 = len(posting1), len(posting2)
            i = j = 0
            while i != len1 and j != len2:
                doc_id1, pp1 = posting1[i]
                doc_id2, pp2 = posting2[j]

                if doc_id1 == doc_id2:
                    positions = [pos2 for pos1 in pp1 for pos2 in pp2 if abs(pos1 - pos2) <= k]
                    if positions:
                        result.append([doc_id1])
                    i += 1
                    j += 1
                elif doc_id1 < doc_id2:
                    i += 1
                else:
                    j += 1
        return result

    def process_proximity_query(self, query):
        """
        Processes a proximity query.
        Arguments:
        - query: String representing the proximity query.
        Returns: Set of document indices satisfying the proximity query.
        """
        proximity = re.findall(r'\d+', query)
        query_terms = query.split()
        token = []
        term = query_terms[0]
        token.append(term)

        operation = "&"
        for i in range(1, len(query_terms)):
            if query_terms[i] == "AND":
                operation = "&"
            elif query_terms[i] == "NOT":
                operation = "!"
            elif "/" in query_terms[i]:
                k = int(proximity[0])
                term1, term2 = token.pop(0), token.pop(0)
                p1 = self.indexer.positional_index.get(term1, [])
                p2 = self.indexer.positional_index.get(term2, [])
                documents = self.positional_intersect(p1, p2, k+1)
                token.append(documents)
            else:
                if operation == "&":
                    term = query_terms[i]
                    token.append(term)
                elif operation == "!":
                    term = query_terms[i]
                    token.append(term)
        return {tuple(doc) for doc in token[0]} if token else set()

class App:
    def __init__(self, root):
        """
        Initializes the App class.
        Arguments:
        - root: Tkinter root window.
        Returns: None
        """
        self.root = root
        self.indexer = DocumentIndexer()
        self.query_processor = QueryProcessor(self.indexer)
        self.setup_gui()

    def setup_gui(self):
        """
        Sets up the graphical user interface.
        Arguments: None
        Returns: None
        """
        self.root.title("Boolean Retrieval System")
        self.root.configure(bg="black")  # Set background color to black

        query_label = tk.Label(self.root, text="Enter your query:",  bg="white", fg="black")  # Set text color to white
        self.query_entry = tk.Text(self.root, height=6, width=60, bg="black", fg="white")  # Set text color to white
        execute_button = tk.Button(self.root, text="Execute Query", command=self.on_query_button_click, bg="red", fg="white")  # Set button colors
        self.result_text = tk.Text(self.root, height=20, width=60, state=tk.DISABLED, bg="black", fg="white")  # Set text color to white
        exit_button = tk.Button(self.root, text="Exit", command=self.exit_app, bg="blue", fg="white")  # Set button colors

        query_label.pack()
        self.query_entry.pack()
        execute_button.pack()
        self.result_text.pack()
        exit_button.pack()

    def on_query_button_click(self):
        """
        Handles the query button click event.
        Arguments: None
        Returns: None
        """
        query = self.query_entry.get("1.0", tk.END)
        self.result_text.config(state=tk.NORMAL)
        self.result_text.delete("1.0", tk.END)

        if '/' in query:
            results = self.query_processor.process_proximity_query(query)
            result_docs = [self.indexer.document_identifiers[doc[0]] for doc in results]
            self.result_text.insert(tk.END, f"Results:\n")
            for doc in result_docs:
                self.result_text.insert(tk.END, f"{doc}\n")
            self.result_text.insert(tk.END, f"\nTotal Documents: {len(result_docs)}\n")
        else:
            results = self.query_processor.process_boolean_query(query)
            result_docs = [self.indexer.document_identifiers[doc] for doc in results]
            self.result_text.insert(tk.END, f"Results:\n")
            for doc in result_docs:
                self.result_text.insert(tk.END, f"{doc}\n")
            self.result_text.insert(tk.END, f"Total Documents: {len(result_docs)}\n")

        self.result_text.config(state=tk.DISABLED)


    def exit_app(self):
        """
        Exits the application.
        Arguments: None
        Returns: None
        """
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()